# Project Memory

## Environment elements

In [21]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY= os.getenv('PINECONE_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
STEAMSHIP_API_KEY = os.getenv('STEAMSHIP_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
GEMINI_KEY = os.getenv('GEMINI_KEY')

os.environ['PATH'] += os.pathsep + '/usr/bin'

## Libraries

In [7]:
# # This notebook requires Tesseract for OCR
%pip install pytesseract

%pip install fpdf

%pip install --upgrade openai

^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Page scanner

In [12]:
from pytesseract import pytesseract

# Page number position
def extract_page_region(image, location='top'):
    height, width = image.shape[:2]
    region = None

    if location == 'top':
        region = image[0:int(height * 0.1), :]  # Top 10% of the image
    elif location == 'bottom':
        region = image[int(height * 0.9):, :]  # Bottom 10% of the image

    return region


# Extract page number & text
def extract_text_with_page_number(image, page_location='top'):
    # Extract the main text
    full_text = pytesseract.image_to_string(image)

    # Extract the page number region if applicable
    if page_location in ['top', 'bottom']:
        page_region = extract_page_region(image, location=page_location)
        page_text = pytesseract.image_to_string(page_region)
    else:
        page_text = ''

    # Extract the page number using regex (look for digits)
    import re
    page_number_match = re.search(r'\b\d+\b', page_text)
    page_number = int(page_number_match.group()) if page_number_match else None

    return full_text, page_number

Create assembled document

In [13]:
# Process Images with Page Location Parameter

def process_images(image_paths, page_location='top'):
    pages = []
    for path in image_paths:
        preprocessed = preprocess_image(path)
        text, page_number = extract_text_with_page_number(preprocessed, page_location=page_location)
        pages.append((page_number, text))

    # Sort pages by number
    pages = sorted(pages, key=lambda x: x[0] if x[0] is not None else float('inf'))
    return [text for _, text in pages]

In [14]:
from fpdf import FPDF

def create_pdf(image_paths, output_path, title="Document", author="Unknown"):
    pdf = FPDF()
    
    # Set metadata
    pdf.set_title(title)
    pdf.set_author(author)
    
    # Add a title page
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.multi_cell(0, 10, f"{title}\nby {author}", align='C')
    
    # Add images as subsequent pages
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(image_path, x=0, y=0, w=210, h=297)  # A4 dimensions in mm
    
    pdf.output(output_path)


Import document

In [15]:
import os

# Image folder location

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    """
    Retrieve all image file paths from a folder.
    
    Args:
        folder_path (str): Path to the folder containing images.
        extensions (tuple): Allowed image file extensions.
        
    Returns:
        list: List of image file paths.
    """
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]


In [16]:
config = {
    'image_folder': '.\Photos\Test set',  # Folder containing images
    'output_text_file': 'output.txt',
    'output_pdf_file': 'output.pdf',  # Optional, only needed if create_pdf=True
    'title': 'Test',
    'author': 'John Doe',
    'page_location': 'bottom',  # Options: 'top', 'bottom', 'none'
    'create_pdf': False,  # Set to True to enable PDF creation
}

# Retrieve image paths from the folder
image_paths = get_image_paths_from_folder(config['image_folder'])

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '.\\Photos\\Test set'

In [15]:
def main(
    image_paths, 
    output_text_file, 
    output_pdf_file=None, 
    title="Document", 
    author="Unknown", 
    page_location="top", 
    create_pdf=False
):
    """
    Main function to process images, extract text, and optionally create a PDF.
    
    Args:
        image_paths (list): List of image file paths.
        output_text_file (str): Path to save the extracted text file.
        output_pdf_file (str): Path to save the PDF (if create_pdf is True).
        title (str): Title of the document.
        author (str): Author of the document.
        page_location (str): Location of page numbers ('top', 'bottom', 'none').
        create_pdf (bool): Whether to create a PDF.
    """
    # Process images and extract text
    texts = process_images(image_paths, page_location=page_location)
    
    # Save text to a file
    save_to_text_file(texts, output_text_file)
    print(f"Text saved to: {output_text_file}")
    
    # Optionally create a PDF
    if create_pdf:
        if not output_pdf_file:
            raise ValueError("output_pdf_file must be provided when create_pdf is True.")
        create_pdf(image_paths, output_pdf_file, title=title, author=author)
        print(f"PDF saved to: {output_pdf_file}")

In [16]:
# Call the main function
main(
    image_paths=image_paths,
    output_text_file=config['output_text_file'],
    output_pdf_file=config['output_pdf_file'],
    title=config['title'],
    author=config['author'],
    page_location=config['page_location'],
    create_pdf=config['create_pdf']
)

NameError: name 'preprocess_image' is not defined

### Annex - Full python script with PDF export

#### tesseract

In [ ]:
import os
import glob
import re
import cv2
from pytesseract import pytesseract
from fpdf import FPDF


# Function to retrieve all image paths from a folder
def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]


# Preprocess image for better OCR results
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh


# Extract the page region based on the location parameter
def extract_page_region(image, location='top'):
    height, width = image.shape[:2]
    if location == 'top':
        return image[0:int(height * 0.1), :]  # Top 10%
    elif location == 'bottom':
        return image[int(height * 0.9):, :]  # Bottom 10%
    return None  # If 'none', skip


# Extract text and page numbers
def extract_text_with_page_number(image, page_location='top'):
    # Full text extraction
    full_text = pytesseract.image_to_string(image)
    
    # Extract page number if applicable
    page_text = ''
    if page_location in ['top', 'bottom']:
        page_region = extract_page_region(image, page_location)
        if page_region is not None:
            page_text = pytesseract.image_to_string(page_region)

    # Find page number using regex
    page_number_match = re.search(r'\b\d+\b', page_text)
    page_number = int(page_number_match.group()) if page_number_match else None

    return full_text, page_number


# Organize pages based on extracted page numbers
def process_images(image_paths, page_location='top'):
    pages = []
    for path in image_paths:
        preprocessed = preprocess_image(path)
        text, page_number = extract_text_with_page_number(preprocessed, page_location=page_location)
        pages.append((page_number, text))
    
    # Sort pages by number
    pages = sorted(pages, key=lambda x: x[0] if x[0] is not None else float('inf'))
    return [text for _, text in pages]


# Save extracted text to a file
def save_to_text_file(texts, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for page_text in texts:
            f.write(page_text + "\n\n")


# Create a PDF with metadata
def create_pdf(image_paths, output_path, title="Document", author="Unknown"):
    pdf = FPDF()
    pdf.set_title(title)
    pdf.set_author(author)

    # Add a title page
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.multi_cell(0, 10, f"{title}\nby {author}", align='C')

    # Add image pages
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(image_path, x=0, y=0, w=210, h=297)  # A4 dimensions in mm

    pdf.output(output_path)


# Main function
def main(
    image_folder,
    output_text_file,
    output_pdf_file=None,
    title="Document",
    author="Unknown",
    page_location="top",
    create_pdf=False
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text
    texts = process_images(image_paths, page_location=page_location)
    
    # Save extracted text to a file
    save_to_text_file(texts, output_text_file)
    print(f"Text saved to: {output_text_file}")
    
    # Optionally create a PDF
    if create_pdf:
        if not output_pdf_file:
            raise ValueError("Output PDF file path must be provided when create_pdf is True.")
        create_pdf(image_paths, output_pdf_file, title=title, author=author)
        print(f"PDF saved to: {output_pdf_file}")


# Configuration
if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test2',  # Replace with your folder path
        'output_text_file': 'output.txt',
        'output_pdf_file': 'output.pdf',  # Optional, only needed if create_pdf=True
        'title': 'Test',
        'author': 'Author Name',
        'page_location': 'bottom',  # Options: 'top', 'bottom', 'none'
        'create_pdf': False,  # Set to True to enable PDF creation
    }

    main(
        image_folder=config['image_folder'],
        output_text_file=config['output_text_file'],
        output_pdf_file=config['output_pdf_file'],
        title=config['title'],
        author=config['author'],
        page_location=config['page_location'],
        create_pdf=config['create_pdf']
    )


Text saved to: output.txt


#### Gemini

In [ ]:
# %pip install google.generativeai

# Install the required packages
%pip install google-generativeai
%pip install --upgrade protobuf

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_python_client-2.159.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached google_generativeai-0.8.4-py3-none-any.whl (175 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
Using cached google_api_python_client-2.159.0-py2.py3-none-any.whl (12.8 MB)
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached uritemplate-4.1.1-py2.p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlepaddle 2.6.2 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 5.29.3 which is incompatible.
paddlepaddle-gpu 2.6.2 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 5.29.3 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import glob
import re
import cv2
from PIL import Image
import google.generativeai as genai

# Replace with your actual API key
genai.configure(api_key=GEMINI_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest") 

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    prompt = f"Extract text from this image: {image_path}"
    generation_config = genai.types.GenerationConfig(
        temperature=0.1,  # Adjust as needed
        max_output_tokens=1024  # Adjust as needed
    )
    response = model.generate_content([prompt, image], generation_config=generation_config)
    return response.text

def process_images(image_paths):
    pages = []
    for path in image_paths:
        text = extract_text_from_image(path)
        pages.append(text)
    return pages

def save_to_text_file(texts, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for page_text in texts:
            f.write(page_text + "\n\n")

def main(
    image_folder,
    output_text_file,
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text
    texts = process_images(image_paths)

    # Save extracted text to a file
    save_to_text_file(texts, output_text_file)
    print(f"Text saved to: {output_text_file}")

if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test2',  # Replace with your folder path
        'output_text_file': 'output.txt',
    }

    main(
        image_folder=config['image_folder'],
        output_text_file=config['output_text_file'],
    )

Text saved to: output.txt


### PaddleOCR

In [ ]:
import os
from fpdf import FPDF
from PIL import Image
import pytesseract


# Function to retrieve all image paths from a folder
def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    """
    Retrieve all image file paths from a folder.

    Args:
        folder_path (str): Path to the folder containing images.
        extensions (tuple): Allowed image file extensions.

    Returns:
        list: List of image file paths.
    """
    if not os.path.exists(folder_path):
        raise ValueError(f"Folder does not exist: {folder_path}")

    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]


def extract_text_tesseract(image_path):
    """
    Extract text from an image using Tesseract OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Extracted text.
    """
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return ""


def process_images_with_tesseract(image_paths):
    """
    Process images and extract text using Tesseract OCR.

    Args:
        image_paths (list): List of image file paths.

    Returns:
        list: Extracted texts in the order of image_paths.
    """
    texts = []
    for path in image_paths:
        print(f"Processing image: {path}")
        text = extract_text_tesseract(path)
        texts.append(text)
    return texts


def save_to_text_file(texts, output_path):
    """
    Save extracted texts to a text file.

    Args:
        texts (list): List of extracted texts.
        output_path (str): Path to the output text file.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for page_text in texts:
            f.write(page_text + "\n\n")


def generate_pdf(image_paths, output_path, title="Document", author="Unknown"):
    """
    Create a PDF from images with a title and author.

    Args:
        image_paths (list): List of image file paths.
        output_path (str): Path to save the PDF file.
        title (str): Title of the document.
        author (str): Author of the document.
    """
    pdf = FPDF()
    pdf.set_title(title)
    pdf.set_author(author)

    # Add a title page
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.multi_cell(0, 10, f"{title}\nby {author}", align='C')

    # Add image pages
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(image_path, x=0, y=0, w=210, h=297)  # A4 dimensions in mm

    pdf.output(output_path)


def main_with_tesseract(
    image_folder,
    output_text_file,
    output_pdf_file=None,
    title="Document",
    author="Unknown",
    generate_pdf_flag=False
):
    """
    Main function to process images, extract text, and optionally create a PDF.

    Args:
        image_folder (str): Path to the folder containing images.
        output_text_file (str): Path to save the extracted text file.
        output_pdf_file (str): Path to save the PDF file (if generate_pdf_flag=True).
        title (str): Title of the document.
        author (str): Author of the document.
        generate_pdf_flag (bool): Whether to create a PDF.
    """
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    texts = process_images_with_tesseract(image_paths)
    save_to_text_file(texts, output_text_file)
    print(f"Text saved to: {output_text_file}")

    if generate_pdf_flag:
        if not output_pdf_file:
            raise ValueError("Output PDF file path must be provided when generate_pdf_flag is True.")
        generate_pdf(image_paths, output_pdf_file, title=title, author=author)
        print(f"PDF saved to: {output_pdf_file}")


if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test2',  # Replace with your folder path
        'output_text_file': 'output_tesseract.txt',
        'output_pdf_file': 'output_tesseract.pdf',  # Optional, only needed if generate_pdf_flag=True
        'title': 'My Book Title',
        'author': 'Author Name',
        'generate_pdf_flag': True,  # Set to True to enable PDF creation
    }

    main_with_tesseract(
        image_folder=config['image_folder'],
        output_text_file=config['output_text_file'],
        output_pdf_file=config['output_pdf_file'],
        title=config['title'],
        author=config['author'],
        generate_pdf_flag=config['generate_pdf_flag']
    )


Processing image: .\Photos\Test_set\20250122_155459.jpg
Processing image: .\Photos\Test_set\20250122_155547.jpg
Processing image: .\Photos\Test_set\20250122_155600.jpg
Processing image: .\Photos\Test_set\20250122_155618.jpg
Processing image: .\Photos\Test_set\20250122_155631.jpg
Processing image: .\Photos\Test_set\20250122_155645.jpg
Processing image: .\Photos\Test_set\20250122_155651.jpg
Text saved to: output_tesseract.txt
PDF saved to: output_tesseract.pdf


In [2]:
%pip install Paddleocr --no-cache-dir
%pip install paddlepaddle
%pip install paddlepaddle-gpu

^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/544.7 kB ? eta -:--:--
   --------------------------------------- 544.7/544.7 kB 17.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 20.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
   --------------------------------------- 948.0/948.0 kB 22.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---- ----------------------------------- 5.8/46.2 MB 27.1 MB/s eta 0:00:02
   ----------- ---------------------------- 13.6/46.2 MB 31.7 MB/s eta 0:00:02
   ----------------- ---------------------- 20.4/46.2 MB 32.3 MB/s eta 0:00:01
   ---------------------- ----------------- 26.5/46.2 MB 31.7 MB/s

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\aurel\\AppData\\Roaming\\Python\\Python312\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [1]:
import os
from paddleocr import PaddleOCR
from fpdf import FPDF


# Initialize PaddleOCR
ocr = PaddleOCR(use_gpu=True, lang='fr')  # Set use_gpu=True if you want to use GPU


# Function to retrieve all image paths from a folder
def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    """
    Retrieve all image file paths from a folder.

    Args:
        folder_path (str): Path to the folder containing images.
        extensions (tuple): Allowed image file extensions.

    Returns:
        list: List of image file paths.
    """
    if not os.path.exists(folder_path):
        raise ValueError(f"Folder does not exist: {folder_path}")

    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]


# Extract text from an image using PaddleOCR
def extract_text_paddleocr(image_path):
    """
    Extract text from an image using PaddleOCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Extracted text.
    """
    try:
        # Perform OCR on the image
        results = ocr.ocr(image_path, cls=True)
        text_lines = [line[1][0] for line in results[0]]  # Extract text from results
        return "\n".join(text_lines)
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return ""


# Process images and extract text
def process_images_with_paddleocr(image_paths):
    """
    Process images and extract text using PaddleOCR.

    Args:
        image_paths (list): List of image file paths.

    Returns:
        list: Extracted texts in the order of image_paths.
    """
    texts = []
    for path in image_paths:
        print(f"Processing image: {path}")
        text = extract_text_paddleocr(path)
        texts.append(text)
    return texts


# Save extracted text to a file
def save_to_text_file(texts, output_path):
    """
    Save extracted texts to a text file.

    Args:
        texts (list): List of extracted texts.
        output_path (str): Path to the output text file.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for page_text in texts:
            f.write(page_text + "\n\n")


# Create a PDF with metadata
def create_pdf(image_paths, output_path, title="Document", author="Unknown"):
    """
    Create a PDF from images with a title and author.

    Args:
        image_paths (list): List of image file paths.
        output_path (str): Path to save the PDF file.
        title (str): Title of the document.
        author (str): Author of the document.
    """
    pdf = FPDF()
    pdf.set_title(title)
    pdf.set_author(author)

    # Add a title page
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.multi_cell(0, 10, f"{title}\nby {author}", align='C')

    # Add image pages
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(image_path, x=0, y=0, w=210, h=297)  # A4 dimensions in mm

    pdf.output(output_path)


# Main function
def main_with_paddleocr(
    image_folder,
    output_text_file,
    output_pdf_file=None,
    title="Document",
    author="Unknown",
    generate_pdf=False  # Renamed parameter to avoid conflict
):
    """
    Main function to process images, extract text, and optionally create a PDF.

    Args:
        image_folder (str): Path to the folder containing images.
        output_text_file (str): Path to save the extracted text file.
        output_pdf_file (str): Path to save the PDF file (if generate_pdf=True).
        title (str): Title of the document.
        author (str): Author of the document.
        generate_pdf (bool): Whether to create a PDF.
    """
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text
    texts = process_images_with_paddleocr(image_paths)
    save_to_text_file(texts, output_text_file)
    print(f"Text saved to: {output_text_file}")

    # Optionally create a PDF
    if generate_pdf:
        if not output_pdf_file:
            raise ValueError("Output PDF file path must be provided when generate_pdf is True.")
        create_pdf(image_paths, output_pdf_file, title=title, author=author)
        print(f"PDF saved to: {output_pdf_file}")


# Configuration
if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test_set',  # Replace with your folder path
        'output_text_file': 'output_paddleocr.txt',
        'output_pdf_file': 'output_paddleocr.pdf',  # Optional, only needed if generate_pdf=True
        'title': 'Test Document',
        'author': 'Author Name',
        'generate_pdf': True,  # Renamed parameter to avoid conflict
    }

    main_with_paddleocr(
        image_folder=config['image_folder'],
        output_text_file=config['output_text_file'],
        output_pdf_file=config['output_pdf_file'],
        title=config['title'],
        author=config['author'],
        generate_pdf=config['generate_pdf']  # Updated parameter name
    )


ModuleNotFoundError: No module named 'paddleocr'

In [16]:
import os
import pandas as pd
from PIL import Image
import google.generativeai as genai
from datetime import datetime
import cv2
import re

# Replace with your actual API key
genai.configure(api_key=GEMINI_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh

def extract_page_region(image_path, location='top'):
    image = Image.open(image_path)
    width, height = image.size
    if location == 'top':
        return image.crop((0, 0, width, int(height * 0.1)))  # Top 10%
    elif location == 'bottom':
        return image.crop((0, int(height * 0.7), width, height))  # Bottom 30%
    return None

def extract_text_and_page_number(image_path, page_location='top'):
    try:
        # Extract full text from the entire image
        image = Image.open(image_path)
        prompt_full = f"Extract the full text from the provided image."
        generation_config = genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=1024
        )
        response_full = model.generate_content([prompt_full, image], generation_config=generation_config)
        full_text = response_full.text

        # Extract page number from the specified region
        page_number = None
        if page_location in ['top', 'bottom']:
            region = extract_page_region(image_path, location=page_location)
            if region:
                prompt_page = f"Identify the page number from this image region."
                response_page = model.generate_content([prompt_page, region], generation_config=generation_config)
                page_text = response_page.text

                # Use regex to find the page number
                page_number_match = re.search(r'\b\d+\b', page_text)
                page_number = int(page_number_match.group()) if page_number_match else None

        return full_text, page_number
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "", None

def process_images(image_paths, override_date=None, page_location='top'):
    pages = []
    for path in image_paths:
        text, page_number = extract_text_and_page_number(path, page_location=page_location)

        # Debugging: Print the extracted text and page number
        print(f"Extracted Text for {path}: {text[:100]}...")  # First 100 characters
        print(f"Page Number: {page_number}")

        # Ensure text is a valid string
        text = str(text).strip() if text else "No text extracted"

        creation_time = datetime.fromtimestamp(os.path.getctime(path)).strftime("%Y-%m-%d %H:%M:%S")
        document_date = override_date if override_date else creation_time

        metadata = {
            "Image Path": path,
            "Document Date": document_date,
            "Extracted Text": text,
            "Page Number": page_number
        }
        pages.append(metadata)
    return pages

def save_to_csv(metadata_list, output_dir, name, author):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file name
    file_name = f"{name}_{author}.csv"
    output_path = os.path.join(output_dir, file_name)

    # Save metadata to a CSV with proper quoting
    df = pd.DataFrame(metadata_list)
    df.to_csv(output_path, index=False, quoting=1)  # quoting=1 ensures proper handling of multiline text
    print(f"CSV saved to: {output_path}")

def main(
    image_folder,
    output_dir,
    name,
    author,
    override_date=None,
    page_location='top'
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text with metadata
    metadata_list = process_images(image_paths, override_date, page_location)

    # Save metadata and text to a CSV file
    save_to_csv(metadata_list, output_dir, name, author)

if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test2',  # Replace with your folder path
        'output_dir': r'.\Family safe',
        'name': "example_name1",  # Replace with the document name
        'author': "example_author",  # Replace with the author name
        'override_date': None,  # Set a specific date string if needed, e.g., "2023-01-01"
        'page_location': 'top',  # Options: 'top', 'bottom', 'none'
    }

    main(
        image_folder=config['image_folder'],
        output_dir=config['output_dir'],
        name=config['name'],
        author=config['author'],
        override_date=config['override_date'],
        page_location=config['page_location']
    )


Extracted Text for .\Photos\Test2\WhatsApp Image 2025-01-14 at 14.41.22.jpeg: C'est le dernier, Isaac, dit Hovel, le plus souvent appelé Louis qui est notre ancêtre direct à la g...
Page Number: 118
CSV saved to: .\Family safe\example_name1_example_author.csv


# Final gemini code

In [17]:
import os
import pandas as pd
from PIL import Image
import google.generativeai as genai
from datetime import datetime
import cv2
import re

# Replace with your actual API key
genai.configure(api_key=GEMINI_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh

def extract_page_region(image_path, location='top'):
    image = Image.open(image_path)
    width, height = image.size
    if location == 'top':
        return image.crop((0, 0, width, int(height * 0.1)))  # Top 10%
    elif location == 'bottom':
        return image.crop((0, int(height * 0.7), width, height))  # Bottom 30%
    return None

def extract_text_and_page_number(image_path, page_location='top'):
    try:
        # Extract full text from the entire image
        image = Image.open(image_path)
        prompt_full = f"Extract the full text from the provided image."
        generation_config = genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=1024
        )
        response_full = model.generate_content([prompt_full, image], generation_config=generation_config)
        full_text = response_full.text

        # Extract page number from the specified region
        page_number = None
        if page_location in ['top', 'bottom']:
            region = extract_page_region(image_path, location=page_location)
            if region:
                prompt_page = f"Identify the page number from this image region."
                response_page = model.generate_content([prompt_page, region], generation_config=generation_config)
                page_text = response_page.text

                # Use regex to find the page number
                page_number_match = re.search(r'\b\d+\b', page_text)
                page_number = int(page_number_match.group()) if page_number_match else None

        return full_text, page_number
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "", None

def process_images(image_paths, override_date=None, page_location='top'):
    pages = []
    for path in image_paths:
        text, page_number = extract_text_and_page_number(path, page_location=page_location)

        # Debugging: Print the extracted text and page number
        print(f"Extracted Text for {path}: {text[:100]}...")  # First 100 characters
        print(f"Page Number: {page_number}")

        # Ensure text is a valid string
        text = str(text).strip() if text else "No text extracted"

        creation_time = datetime.fromtimestamp(os.path.getctime(path)).strftime("%Y-%m-%d %H:%M:%S")
        document_date = override_date if override_date else creation_time

        metadata = {
            "Image Path": path,
            "Document Date": document_date,
            "Extracted Text": text,
            "Page Number": page_number
        }
        pages.append(metadata)
    return pages

def save_to_csv(metadata_list, output_dir, name, author):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file name
    file_name = f"{name}_{author}.csv"
    output_path = os.path.join(output_dir, file_name)

    # Save metadata to a CSV with proper quoting
    df = pd.DataFrame(metadata_list)
    df.to_csv(output_path, index=False, quoting=1)  # quoting=1 ensures proper handling of multiline text
    print(f"CSV saved to: {output_path}")

def main(
    image_folder,
    output_dir,
    name,
    author,
    override_date=None,
    page_location='top'
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text with metadata
    metadata_list = process_images(image_paths, override_date, page_location)

    # Save metadata and text to a CSV file
    save_to_csv(metadata_list, output_dir, name, author)

if __name__ == "__main__":
    config = {
        'image_folder': r'.\Photos\Test_set',  # Replace with your folder path
        'output_dir': r'.\Family safe',
        'name': "test",  # Replace with the document name
        'author': "test",  # Replace with the author name
        'override_date': None,  # Set a specific date string if needed, e.g., "2023-01-01"
        'page_location': 'bottom',  # Options: 'top', 'bottom', 'none'
    }

    main(
        image_folder=config['image_folder'],
        output_dir=config['output_dir'],
        name=config['name'],
        author=config['author'],
        override_date=config['override_date'],
        page_location=config['page_location']
    )


Extracted Text for .\Photos\Test_set\20250122_155459.jpg: Il était 0 h 7. Le chien était allongé dans l'herbe au milieu de la pelouse, devant chez Mme Shears....
Page Number: None
Extracted Text for .\Photos\Test_set\20250122_155547.jpg: En fait, le livre n'était pas terminé, parce que
5 jours plus tard, j'ai vu 5 voitures rouges d'affi...
Page Number: None
Extracted Text for .\Photos\Test_set\20250122_155600.jpg: J'ai dit : « Quel jour ? »
Elle a dit : « Quand je suis revenue, tu n'étais
plus là. J'ai dû manger ...
Page Number: None
Error processing .\Photos\Test_set\20250122_155618.jpg: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.
Extracted Text for .\Photos\Test_set\20250122_155618.jpg: ...
Page Number: None
Extracted Text for .\Photos\Test_set\20

# Gemini Json version 

In [23]:
import os
import pandas as pd
import json
from PIL import Image
import google.generativeai as genai
from datetime import datetime
import cv2
import re

# Replace with your actual API key
genai.configure(api_key=GEMINI_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh

def extract_page_region(image_path, location='top'):
    image = Image.open(image_path)
    width, height = image.size
    if location == 'top':
        return image.crop((0, 0, width, int(height * 0.1)))  # Top 10%
    elif location == 'bottom':
        return image.crop((0, int(height * 0.9), width, height))  # Bottom 10%
    return None

def extract_text_and_page_number(image_path, page_location='top'):
    try:
        # Extract full text from the entire image
        image = Image.open(image_path)
        prompt_full = f"Extract the full text from the provided image."
        generation_config = genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=1024
        )
        response_full = model.generate_content([prompt_full, image], generation_config=generation_config)
        full_text = response_full.text

        # Extract page number from the specified region
        page_number = None
        if page_location in ['top', 'bottom']:
            region = extract_page_region(image_path, location=page_location)
            if region:
                prompt_page = f"Identify the page number from this image region."
                response_page = model.generate_content([prompt_page, region], generation_config=generation_config)
                page_text = response_page.text

                # Use regex to find the page number
                page_number_match = re.search(r'\b\d+\b', page_text)
                page_number = int(page_number_match.group()) if page_number_match else None

        return full_text, page_number
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "", None

def process_images(image_paths, page_location='top'):
    pages = []
    for path in image_paths:
        text, page_number = extract_text_and_page_number(path, page_location=page_location)

        # Debugging: Print the extracted text and page number
        print(f"Extracted Text for {path}: {text[:100]}...")  # First 100 characters
        print(f"Page Number: {page_number}")

        # Ensure text is a valid string
        text = str(text).strip() if text else "No text extracted"

        metadata = {
            "Image Path": path,
            "Extracted Text": text,
            "Page Number": page_number
        }
        pages.append(metadata)
    return pages

def save_to_json(metadata_list, output_dir, name, author):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file name
    file_name = f"{name}_{author}.json"
    output_path = os.path.join(output_dir, file_name)

    # Add document-level metadata
    document_metadata = {
        "Author": author,
        "Name": name,
        "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Pages": metadata_list
    }

    # Save metadata to a JSON file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(document_metadata, json_file, indent=4, ensure_ascii=False)
    print(f"JSON saved to: {output_path}")

def main(
    image_folder,
    output_dir,
    name,
    author,
    page_location='top'
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text with metadata
    metadata_list = process_images(image_paths, page_location)

    # Save metadata and text to a JSON file
    save_to_json(metadata_list, output_dir, name, author)

if __name__ == "__main__":
    config = {
        'image_folder': r'C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Photos\test pdf\ilovepdf_pages-to-jpg',  # Replace with your folder path
        'output_dir': r'C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Family safe',
        'name': "Pdf img",  # Replace with the document name
        'author': "John Doe",  # Replace with the author name
        'page_location': 'top',  # Options: 'top', 'bottom', 'none'
    }

    main(
        image_folder=config['image_folder'],
        output_dir=config['output_dir'],
        name=config['name'],
        author=config['author'],
        page_location=config['page_location']
    )


Extracted Text for C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Photos\test pdf\ilovepdf_pages-to-jpg\2412.02210v3_page-0001.jpg: CC-OCR: A Comprehensive and Challenging OCR Benchmark for Evaluating
Large Multimodal Models in Lite...
Page Number: None
Extracted Text for C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Photos\test pdf\ilovepdf_pages-to-jpg\2412.02210v3_page-0002.jpg: only includes reading plain texts but also involves position grounding, structured layouts, and hand...
Page Number: None
Extracted Text for C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Photos\test pdf\ilovepdf_pages-to-jpg\2412.02210v3_page-0003.jpg: ---
LMMS
Structured inputs and outputs
Fine-grained visual challenges
Orientation
Kitchen
Grounding
...
Page Number: None
Extracted Text for C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Photos\test pdf\ilovepdf_pages-to-jpg\2412.02210v3_page-0004.jpg: TextMonkey
KOSMOS2.5-20.9
Total
Multi-Scene OCR
Multilingual O

# Tes define type

In [29]:
import os
import pandas as pd
import json
from PIL import Image
import google.generativeai as genai
from datetime import datetime
import cv2
import re

# Replace with your actual API key
genai.configure(api_key=GEMINI_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")

def get_image_paths_from_folder(folder_path, extensions=('.jpg', '.jpeg', '.png')):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.lower().endswith(extensions)
    ]

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh

def extract_page_region(image_path, location='top'):
    image = Image.open(image_path)
    width, height = image.size
    if location == 'top':
        return image.crop((0, 0, width, int(height * 0.1)))  # Top 10%
    elif location == 'bottom':
        return image.crop((0, int(height * 0.9), width, height))  # Bottom 10%
    return None

def extract_text_and_page_number(image_path, page_location='top'):
    try:
        # Extract full text from the entire image
        image = Image.open(image_path)
        prompt_full = f"Extract the full text from the provided image."
        generation_config = genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=1024
        )
        response_full = model.generate_content([prompt_full, image], generation_config=generation_config)
        full_text = response_full.text

        # Extract page number from the specified region
        page_number = None
        if page_location in ['top', 'bottom']:
            region = extract_page_region(image_path, location=page_location)
            if region:
                prompt_page = f"Identify the page number from this image region."
                response_page = model.generate_content([prompt_page, region], generation_config=generation_config)
                page_text = response_page.text

                # Use regex to find the page number
                page_number_match = re.search(r'\b\d+\b', page_text)
                page_number = int(page_number_match.group()) if page_number_match else None

        return full_text, page_number
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "", None

def process_images(image_paths, page_location='top'):
    pages = []
    for path in image_paths:
        text, page_number = extract_text_and_page_number(path, page_location=page_location)

        # Debugging: Print the extracted text and page number
        print(f"Extracted Text for {path}: {text[:100]}...")  # First 100 characters
        print(f"Page Number: {page_number}")

        # Ensure text is a valid string
        text = str(text).strip() if text else "No text extracted"

        metadata = {
            "Image Path": path,
            "Extracted Text": text,
            "Page Number": page_number
        }
        pages.append(metadata)
    return pages

def save_to_json(metadata_list, output_dir, name, author, type):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file name
    file_name = f"{name}_{author}.json"
    output_path = os.path.join(output_dir, file_name)

    # Add document-level metadata
    document_metadata = {
        "Author": author,
        "Name": name,
        "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Pages": metadata_list,
        "Type": type
    }

    # Save metadata to a JSON file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(document_metadata, json_file, indent=4, ensure_ascii=False)
    print(f"JSON saved to: {output_path}")

def main(
    image_folder,
    output_dir,
    name,
    author,
    page_location='top'
):
    # Retrieve image paths
    image_paths = get_image_paths_from_folder(image_folder)
    if not image_paths:
        print(f"No images found in folder: {image_folder}")
        return

    # Process images and extract text with metadata
    metadata_list = process_images(image_paths, page_location)

    # Save metadata and text to a JSON file
    save_to_json(metadata_list, output_dir, name, author, type=config['type'])

if __name__ == "__main__":
    config = {
        'image_folder': r'Photos\la belle',  # Replace with your folder path
        'output_dir': r'C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Family safe',
        'name': "La belle au bois dormant",  # Replace with the document name
        'author': "Perrault",  # Replace with the author name
        'page_location': 'bottom',  # Options: 'top', 'bottom', 'none'
        'type': 'scan'
    }

    main(
        image_folder=config['image_folder'],
        output_dir=config['output_dir'],
        name=config['name'],
        author=config['author'],
        page_location=config['page_location']
    )


Extracted Text for Photos\la belle\20250124_120543.jpg: LA BELLE AU BOIS DORMANT

Il était une fois un roi et une reine qui étaient si fâchés de n'avoir poi...
Page Number: None
Extracted Text for Photos\la belle\20250124_120558.jpg: LA BELLE AU BOIS DORMANT

La plus jeune lui donna pour don qu'elle serait la plus belle personne du ...
Page Number: None
Extracted Text for Photos\la belle\20250124_120606.jpg: ---
LA BELLE AU BOIS DORMANT

— Que faites-vous là, ma bonne femme ? dit la princesse.
— Je file, ma...
Page Number: None
Extracted Text for Photos\la belle\20250124_120607.jpg: LA BELLE AU BOIS DORMANT

— Que faites-vous là, ma bonne femme ? dit la princesse.
— Je file, ma bel...
Page Number: None
JSON saved to: C:\Users\aurel\OneDrive\Documents\Python\IronHack\Project\Family safe\La belle au bois dormant_Perrault.json
